In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
DATA_DIR_PATH = os.path.join('..', '..', 'data')

def laod_data(file_name):
  file_path = os.path.join(DATA_DIR_PATH, file_name)
  return pd.read_csv(file_path, header=0)

In [3]:
df_train = laod_data('bookmark_train_data.csv')
print(len(df_train))
#df_train.head(3)

199


In [4]:
from sklearn.preprocessing import LabelEncoder

cat_le = LabelEncoder()
cat_le.fit(df_train['category'])

WORD_CLASSE_LABELS = ['その他', 'フィラー', '副詞', '助動詞', '助詞',
                     '動詞', '名詞', '形容詞', '感動詞', '接続詞',
                     '接頭詞', '記号', '連体詞',]
new_wc_columns = {}
for (i, name) in enumerate(WORD_CLASSE_LABELS):
  new_wc_columns['wc_sum_{}'.format(name)] = 'wc_sum_{}'.format(i)
  new_wc_columns['wc_rate_{}'.format(name)] = 'wc_rate_{}'.format(i)


def data_preprocess(df):
  new_df = df.drop(['ncode', 'title', 'bookmark_cat'], axis=1)

  new_df['category'] = cat_le.transform(new_df['category'])

  new_df['rating'] = new_df['rating'] * 10
  new_df = new_df.astype({'rating': 'int32'})

  #wc_columns = [col for col in new_df.columns if col.find('wc_') != -1]
  #new_df = new_df.drop(wc_columns, axis=1)
  wc_columns = [col for col in new_df.columns if col.find('wc_sum_') != -1]
  new_df = new_df.drop(wc_columns, axis=1)
  new_df = new_df.rename(columns=new_wc_columns)

  kw_columns = [col for col in new_df.columns if col.find('kw_') != -1]
  new_df = new_df.drop(kw_columns, axis=1)

  return new_df

In [5]:
df_train.keys()

Index(['ncode', 'title', 'category', 'bookmark_cat', 'created_at',
       'updated_at', 'char_count_sum', 'new_line_count_sum',
       'talk_char_count_sum', 'new_line_count_rate_sum',
       'talk_char_count_rate_sum', 'char_count_avg', 'new_line_count_avg',
       'talk_char_count_avg', 'new_line_count_rate_avg',
       'talk_char_count_rate_avg', 'wc_sum_その他', 'wc_sum_フィラー', 'wc_sum_副詞',
       'wc_sum_助動詞', 'wc_sum_助詞', 'wc_sum_動詞', 'wc_sum_名詞', 'wc_sum_形容詞',
       'wc_sum_感動詞', 'wc_sum_接続詞', 'wc_sum_接頭詞', 'wc_sum_記号', 'wc_sum_連体詞',
       'wc_rate_その他', 'wc_rate_フィラー', 'wc_rate_副詞', 'wc_rate_助動詞',
       'wc_rate_助詞', 'wc_rate_動詞', 'wc_rate_名詞', 'wc_rate_形容詞', 'wc_rate_感動詞',
       'wc_rate_接続詞', 'wc_rate_接頭詞', 'wc_rate_記号', 'wc_rate_連体詞', 'kw_冒険者',
       'kw_魔王', 'kw_日常', 'kw_成り上がり', 'kw_内政', 'kw_恋愛', 'kw_無双', 'kw_青春',
       'kw_ハーレム', 'kw_男主人公', 'kw_奴隷', 'kw_テンプレ', 'kw_勘違い', 'kw_ほのぼの', 'kw_貴族',
       'kw_ギルド', 'kw_コメディ', 'kw_異世界', 'kw_エルフ', 'kw_ファンタジー', 'kw_オリジナル戦記',
       

In [6]:
df_train2 = data_preprocess(df_train)
df_train2.head(3)

,category,created_at,updated_at,char_count_sum,new_line_count_sum,talk_char_count_sum,new_line_count_rate_sum,talk_char_count_rate_sum,char_count_avg,new_line_count_avg,...,wc_rate_4,wc_rate_5,wc_rate_6,wc_rate_7,wc_rate_8,wc_rate_9,wc_rate_10,wc_rate_11,wc_rate_12,rating
0,6,1.586338e+09,1.593468e+09,92340,3840,28950,0.041585,0.313515,3078.0,128.0,...,0.237664,0.226083,0.225076,0.013595,0.006042,0.007049,0.006042,0.144008,0.011078,30
1,6,1.590714e+09,1.593338e+09,77880,4140,4950,0.053159,0.063559,2596.0,138.0,...,0.238590,0.238013,0.217215,0.017909,0.001155,0.010399,0.001155,0.138070,0.013287,40
2,6,1.575176e+09,1.593429e+09,59880,3090,10230,0.051603,0.170842,1996.0,103.0,...,0.240486,0.177328,0.285020,0.014575,0.000000,0.005668,0.008907,0.148178,0.005668,30


In [7]:
X = df_train2.drop(['rating'], axis=1).values
y = df_train2['rating'].values
print(X.shape, y.shape)

(199, 26) (199,)


In [8]:
feature_names = df_train2.keys()[:-1]

print(feature_names)
print(len(feature_names))

Index(['category', 'created_at', 'updated_at', 'char_count_sum',
       'new_line_count_sum', 'talk_char_count_sum', 'new_line_count_rate_sum',
       'talk_char_count_rate_sum', 'char_count_avg', 'new_line_count_avg',
       'talk_char_count_avg', 'new_line_count_rate_avg',
       'talk_char_count_rate_avg', 'wc_rate_0', 'wc_rate_1', 'wc_rate_2',
       'wc_rate_3', 'wc_rate_4', 'wc_rate_5', 'wc_rate_6', 'wc_rate_7',
       'wc_rate_8', 'wc_rate_9', 'wc_rate_10', 'wc_rate_11', 'wc_rate_12'],
      dtype='object')
26


In [12]:
#from sklearn.model_selection import train_test_split

#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=1)
#X_train, X_val, y_train, y_val = train_test_split(X_resampled, y_resampled, test_size=0.3, shuffle=True, random_state=1)

#print(X_train.shape, y_train.shape)
#print(X_val.shape, y_val.shape)

In [13]:
from sklearn.cluster import KMeans

In [14]:
model = KMeans(n_clusters=3)

In [15]:
model.fit(X)

KMeans(n_clusters=3)

In [16]:
cluster = model.predict(X)
cluster

array([1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1,
       1, 2, 2, 1, 1, 1, 1, 2, 1, 1, 0, 1, 1, 1, 0, 1, 2, 2, 1, 1, 2, 1,
       2, 1, 2, 2, 1, 2, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 2, 0, 2, 0, 0, 2, 0, 2, 0, 0, 0, 0,
       0, 2, 2, 2, 1, 2, 2, 0, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 2, 2,
       2, 1, 1, 1, 0, 0, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 0, 1, 2, 2, 1, 1, 2, 1, 2, 2, 2, 1, 2, 1,
       0, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 0, 0, 2, 2, 2, 0, 0, 2, 0, 2, 2,
       2], dtype=int32)

In [20]:
df = pd.DataFrame(X, columns=feature_names)
df['cluster'] = cluster

In [31]:
df.head(15)

,category,created_at,updated_at,char_count_sum,new_line_count_sum,talk_char_count_sum,new_line_count_rate_sum,talk_char_count_rate_sum,char_count_avg,new_line_count_avg,...,wc_rate_4,wc_rate_5,wc_rate_6,wc_rate_7,wc_rate_8,wc_rate_9,wc_rate_10,wc_rate_11,wc_rate_12,cluster
0,6.0,1.586338e+09,1.593468e+09,92340.0,3840.0,28950.0,0.041585,0.313515,3078.0,128.0,...,0.237664,0.226083,0.225076,0.013595,0.006042,0.007049,0.006042,0.144008,0.011078,1
1,6.0,1.590714e+09,1.593338e+09,77880.0,4140.0,4950.0,0.053159,0.063559,2596.0,138.0,...,0.238590,0.238013,0.217215,0.017909,0.001155,0.010399,0.001155,0.138070,0.013287,1
2,6.0,1.575176e+09,1.593429e+09,59880.0,3090.0,10230.0,0.051603,0.170842,1996.0,103.0,...,0.240486,0.177328,0.285020,0.014575,0.000000,0.005668,0.008907,0.148178,0.005668,1
3,6.0,1.482992e+09,1.592779e+09,88200.0,7770.0,11070.0,0.088095,0.125510,2940.0,259.0,...,0.216523,0.219222,0.206803,0.010799,0.012959,0.007559,0.001080,0.203024,0.003780,2
4,6.0,1.589543e+09,1.593688e+09,100920.0,3000.0,34290.0,0.029727,0.339774,3364.0,100.0,...,0.264463,0.205307,0.279252,0.015224,0.001305,0.008264,0.001305,0.098304,0.011309,1
5,6.0,1.585669e+09,1.593432e+09,198180.0,6540.0,87390.0,0.033000,0.440963,6606.0,218.0,...,0.241278,0.205936,0.227458,0.013820,0.005890,0.006797,0.000906,0.161078,0.008382,1
6,6.0,1.572513e+09,1.593421e+09,78840.0,4620.0,1410.0,0.058600,0.017884,2628.0,154.0,...,0.245772,0.245209,0.227170,0.010710,0.001691,0.004510,0.000000,0.114994,0.007328,1
7,6.0,1.584934e+09,1.593508e+09,51300.0,1980.0,3450.0,0.038596,0.067251,1710.0,66.0,...,0.242481,0.217105,0.250000,0.015977,0.001880,0.008459,0.001880,0.139098,0.004699,1
8,4.0,1.447730e+09,1.547568e+09,40080.0,1590.0,0.0,0.039671,0.000000,1336.0,53.0,...,0.244014,0.238312,0.257697,0.022805,0.003421,0.003421,0.001140,0.090080,0.010262,2
9,6.0,1.582863e+09,1.593486e+09,42750.0,1950.0,3090.0,0.045614,0.072281,1425.0,65.0,...,0.274576,0.215819,0.277966,0.012429,0.000000,0.009040,0.000000,0.093785,0.006780,1


In [32]:
df_results = pd.DataFrame()

#df_results['group0'] = round(df[df['cluster'] == 0].mean().astype(float))
#df_results['group1'] = round(df[df['cluster'] == 1].mean().astype(float))
#df_results['group2'] = round(df[df['cluster'] == 2].mean().astype(float))
df_results['group0'] = df[df['cluster'] == 0].mean()
df_results['group1'] = df[df['cluster'] == 1].mean()
df_results['group2'] = df[df['cluster'] == 2].mean()

df_results.set_index(df.columns, inplace=True)

In [40]:
#np.set_printoptions(suppress=True)
pd.options.display.float_format = '{:.6f}'.format

df_results

,group0,group1,group2
category,6.317073,6.222222,5.911765
created_at,1392036333.658537,1554619575.333333,1468062828.529412
updated_at,1526728519.024390,1586277896.000000,1572476560.588235
char_count_sum,134320.243902,115828.000000,122449.852941
new_line_count_sum,5693.414634,5439.333333,5609.117647
talk_char_count_sum,16493.414634,23026.000000,16835.294118
new_line_count_rate_sum,0.046871,0.047185,0.047151
talk_char_count_rate_sum,0.085689,0.186484,0.119301
char_count_avg,4477.341463,3860.933333,4081.661765
new_line_count_avg,189.780488,181.311111,186.970588


In [41]:
df_results.drop(['cluster']).T

,category,created_at,updated_at,char_count_sum,new_line_count_sum,talk_char_count_sum,new_line_count_rate_sum,talk_char_count_rate_sum,char_count_avg,new_line_count_avg,...,wc_rate_3,wc_rate_4,wc_rate_5,wc_rate_6,wc_rate_7,wc_rate_8,wc_rate_9,wc_rate_10,wc_rate_11,wc_rate_12
group0,6.317073,1392036333.658537,1526728519.024390,134320.243902,5693.414634,16493.414634,0.046871,0.085689,4477.341463,189.780488,...,0.075517,0.242022,0.204961,0.270560,0.015511,0.002578,0.006041,0.002704,0.133974,0.008429
group1,6.222222,1554619575.333333,1586277896.000000,115828.000000,5439.333333,23026.000000,0.047185,0.186484,3860.933333,181.311111,...,0.079151,0.238558,0.207160,0.255555,0.014388,0.003905,0.006647,0.002521,0.144981,0.008670
group2,5.911765,1468062828.529412,1572476560.588235,122449.852941,5609.117647,16835.294118,0.047151,0.119301,4081.661765,186.970588,...,0.077772,0.239714,0.209744,0.259776,0.015307,0.003224,0.005613,0.002202,0.139881,0.008079
